

<a href="https://colab.research.google.com/github/rctejon/linear-regression-quiz/blob/master/quiz3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns

from pandas_profiling import ProfileReport

### Reading the dataset

In [3]:
medical_df = pd.read_csv('https://raw.githubusercontent.com/rctejon/linear-regression-quiz/master/data/medical.csv', sep = ',')

In [4]:
medical_df.shape

(1338, 7)

In [5]:
medical_df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [6]:
medical_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [7]:
medical_df['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [8]:
medical_df['sex'] = medical_df['sex'].replace('female', 0)
medical_df['sex'] = medical_df['sex'].replace('male', 1)
medical_df['smoker'] = medical_df['smoker'].replace('yes', 1)
medical_df['smoker'] = medical_df['smoker'].replace('no', 0)
medical_df['region'] = medical_df['region'].replace('northeast', 0)
medical_df['region'] = medical_df['region'].replace('southeast', 1)
medical_df['region'] = medical_df['region'].replace('southwest', 2)
medical_df['region'] = medical_df['region'].replace('northwest', 3)

In [9]:
features = ['age', 'sex', 'bmi', 'children', 'smoker', 'region']

### Splitting train and test datasets

In [10]:
X = medical_df[features]
Y = medical_df['charges']

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

In [12]:
X_train.shape

(1070, 6)

In [13]:
X_test.shape

(268, 6)

### Training the model

In [14]:
regr = LinearRegression()

In [15]:
regr.fit(X_train, Y_train)

LinearRegression()

In [16]:
regr.coef_

array([  258.35331385,  -240.66580348,   304.14035685,   413.00447469,
       23758.70488428,  -108.11651622])

In [17]:
regr.intercept_

-11228.8481148371

Entrene un primer modelo de regresión lineal sin aplicar ningún tipo de regularización. Evalúe dicho modelo y
concluya:

  1. ¿Es aceptable el error obtenido?
  2. ¿Hay evidencia de overfitting?

In [18]:
preds_train = regr.predict(X_train)
preds_test = regr.predict(X_test)

In [19]:
np.sqrt(mean_squared_error(Y_train, preds_train)), np.sqrt(mean_squared_error(Y_test, preds_test))

(6077.429016192672, 5974.390555035169)

**Respuestas**
1. El error es exagerado debido a que el MSE nos indica que en promedio nos equivocamos alrededor de 6000 dolares en nuestras predicciones, lo cual para predecir precios de alrededor de 4 o 5 grados de magnitud modificaria considerablemente el valor real.

2. No existe evidencia de overfitting debido a que ambos errores de testing y de training son muy altos, por lo tanto hay overfitting.

Aplique una transformación polinomial a los datos de entrada y regularización Ridge o Lasso al modelo de
regresión. Pruebe con al menos 2 grados diferentes del polinomio y con al menos 3 valores de alpha para la
regularización. Evalúe dichos modelos y concluya:
1. ¿Fue posible mejorar el error? ¿Qué hiper-parámetros tiene el modelo que produce el menor error?
2. ¿Qué atributos parecen ser los más importantes para realizar la predicción?

Primero aplicamos una transformación polinomial con grado 2 en los features 

In [20]:
from sklearn.preprocessing import PolynomialFeatures

X_poly_2 = PolynomialFeatures(degree=2, include_bias=False).fit_transform(X)
regr_poly_2 = LinearRegression()

X_train_poly_2, X_test_poly_2, Y_train_poly_2, Y_test_poly_2 = train_test_split(X_poly_2, Y, test_size = 0.2, random_state = 1)

regr_poly_2.fit(X_train_poly_2, Y_train_poly_2)

LinearRegression()

In [21]:
regr_poly_2.coef_

array([-1.23636980e+02, -1.05736276e+03,  4.54741676e+02,  1.52687241e+03,
       -9.59511731e+03, -1.86740667e+03,  3.79239737e+00,  2.67048692e+01,
        2.21249241e+00,  3.40916152e+00, -2.04142501e+01,  1.37649274e+00,
       -1.05736276e+03,  1.79885235e+01, -2.18910558e+02,  5.39655455e+01,
        5.80691879e+01, -8.46110009e+00, -1.73317779e+01,  1.44493711e+03,
        7.84343052e+00, -1.05681505e+02, -4.69261994e+02,  3.60631139e+01,
       -9.59511731e+03, -3.40412117e+00,  4.41481173e+02])

In [22]:
regr_poly_2.intercept_

-1540.709964913507

In [23]:
preds_train_poly_2 = regr_poly_2.predict(X_train_poly_2)
preds_test_poly_2 = regr_poly_2.predict(X_test_poly_2)

In [24]:
np.sqrt(mean_squared_error(Y_train_poly_2, preds_train_poly_2)), np.sqrt(mean_squared_error(Y_test_poly_2, preds_test_poly_2))

(4808.76702659995, 4567.397048868338)

Debido a que el error sigue tan grande se probara con una transformación polinomial de grado 5

In [25]:
X_poly_10 = PolynomialFeatures(degree=10, include_bias=False).fit_transform(X)
regr_poly_10 = LinearRegression()

X_train_poly_10, X_test_poly_10, Y_train_poly_10, Y_test_poly_10 = train_test_split(X_poly_10, Y, test_size = 0.2, random_state = 1)

regr_poly_10.fit(X_train_poly_10, Y_train_poly_10)

LinearRegression()

In [26]:
regr_poly_10.coef_

array([-2.43534636e-01, -3.19889924e-02,  2.06815632e-01, ...,
       -9.27881686e-07, -1.78430008e-06, -1.36867280e-01])

In [27]:
regr_poly_10.intercept_

70240.6749000129

In [28]:
preds_train_poly_10 = regr_poly_10.predict(X_train_poly_10)
preds_test_poly_10 = regr_poly_10.predict(X_test_poly_10)

In [29]:
np.sqrt(mean_squared_error(Y_train_poly_10, preds_train_poly_10)), np.sqrt(mean_squared_error(Y_test_poly_10, preds_test_poly_10))

(1710.1262276110504, 8744843.009838318)

En este caso se ve que el error de training disminuyo considerablemente pero el test aumento un monton, por lo tanto estamos hablando de overfitting por lo cual tiene sentido hacer una regularización. Empezaremos con una lasso con un alpha de 0.01, luego una de 0.1 y por ultimo uno de 1

Alpha 0.01

In [81]:
X_train_poly_L1_1, X_test_poly_L1_1, Y_train_poly_L1_1, Y_test_poly_L1_1 = train_test_split(X_poly_10, Y, test_size = 0.2, random_state = 1)

regr_poly_L1_1 = SGDRegressor(penalty='L1', alpha=0.01 / X_train_poly_L1_1.shape[0], tol=None, max_iter=1000)

regr_poly_L1_1.fit(X_train_poly_L1_1, Y_train_poly_L1_1)

SGDRegressor(alpha=9.345794392523365e-06, penalty='L1', tol=None)

In [82]:
regr_poly_L1_1.coef_

array([5.16399637e+13, 1.60905232e+11, 3.70098330e+13, ...,
       1.04447897e+14, 2.94465419e+14, 4.33112703e+16])

In [83]:
regr_poly_L1_1.intercept_

array([1.62064917e+12])

In [84]:
preds_train_poly_L1_1 = regr_poly_L1_1.predict(X_train_poly_L1_1)
preds_test_poly_L1_1 = regr_poly_L1_1.predict(X_test_poly_L1_1)

In [85]:
np.sqrt(mean_squared_error(Y_train_poly_L1_1, preds_train_poly_L1_1)), np.sqrt(mean_squared_error(Y_test_poly_L1_1, preds_test_poly_L1_1))

(5.100385392291315e+42, 5.0074592640811944e+42)

Alpha = 0.1

In [86]:
X_train_poly_L1_2, X_test_poly_L1_2, Y_train_poly_L1_2, Y_test_poly_L1_2 = train_test_split(X_poly_10, Y, test_size = 0.2, random_state = 1)

regr_poly_L1_2 = SGDRegressor(penalty='L1', alpha=0.1 / X_train_poly_L1_2.shape[0], tol=None, max_iter=1000)

regr_poly_L1_2.fit(X_train_poly_L1_2, Y_train_poly_L1_2)

SGDRegressor(alpha=9.345794392523365e-05, penalty='L1', tol=None)

In [87]:
regr_poly_L1_2.coef_

array([-5.47357643e+13, -1.64891213e+12, -5.52299340e+13, ...,
       -1.23719566e+15, -3.68214915e+15, -4.78722726e+16])

In [88]:
regr_poly_L1_2.intercept_

array([-1.9916322e+12])

In [89]:
preds_train_poly_L1_2 = regr_poly_L1_2.predict(X_train_poly_L1_2)
preds_test_poly_L1_2 = regr_poly_L1_2.predict(X_test_poly_L1_2)

In [90]:
np.sqrt(mean_squared_error(Y_train_poly_L1_2, preds_train_poly_L1_2)), np.sqrt(mean_squared_error(Y_test_poly_L1_2, preds_test_poly_L1_2))

(1.1018424941741497e+44, 1.113973152770239e+44)

Alpha = 1

In [91]:
X_train_poly_L1_3, X_test_poly_L1_3, Y_train_poly_L1_3, Y_test_poly_L1_3 = train_test_split(X_poly_10, Y, test_size = 0.2, random_state = 1)

regr_poly_L1_3 = SGDRegressor(penalty='L1', alpha=1 / X_train_poly_L1_3.shape[0], tol=None, max_iter=1000)

regr_poly_L1_3.fit(X_train_poly_L1_3, Y_train_poly_L1_3)

SGDRegressor(alpha=0.0009345794392523365, penalty='L1', tol=None)

In [77]:
regr_poly_L1_3.coef_

array([3.88261499e+13, 1.28696243e+12, 3.06761447e+13, ...,
       1.11723804e+15, 3.33915577e+15, 1.25357495e+16])

In [78]:
regr_poly_L1_3.intercept_

array([1.48633223e+12])

In [79]:
preds_train_poly_L1_3 = regr_poly_L1_3.predict(X_train_poly_L1_3)
preds_test_poly_L1_3 = regr_poly_L1_3.predict(X_test_poly_L1_3)

In [80]:
np.sqrt(mean_squared_error(Y_train_poly_L1_3, preds_train_poly_L1_3)), np.sqrt(mean_squared_error(Y_test_poly_L1_2, preds_test_poly_L1_3))

(6.88128671094069e+43, 6.970921705970118e+43)

En ninguno de estos casos funciono y el error se se aumento exageradamente, ahora se probara con regularuzación L2 con los mismos alphas, ademas se usara la transformación polinomial de grado 2


Alpha 0.01

In [92]:
X_train_poly_L2_1, X_test_poly_L2_1, Y_train_poly_L2_1, Y_test_poly_L2_1 = train_test_split(X_poly_2, Y, test_size = 0.2, random_state = 1)

regr_poly_L2_1 = SGDRegressor(penalty='L2', alpha=0.01 / X_train_poly_L2_1.shape[0], tol=None, max_iter=1000)

regr_poly_L2_1.fit(X_train_poly_L2_1, Y_train_poly_L2_1)

SGDRegressor(alpha=9.345794392523365e-06, penalty='L2', tol=None)

In [93]:
regr_poly_L2_1.coef_

array([-7.56991664e+10,  2.59154442e+10, -1.67702217e+11,  5.47138067e+10,
        6.16407568e+10,  1.43126759e+11,  5.83694199e+11, -1.37977680e+11,
       -5.94783569e+11, -3.35711939e+11, -1.60771498e+11, -3.55193565e+11,
        2.59154442e+10,  1.30890413e+09, -4.18728602e+10,  9.62086980e+10,
        4.53503278e+11,  5.92790531e+10, -3.55978785e+11, -3.80983338e+11,
        4.43703531e+11, -7.72223265e+11,  1.60607730e+11, -7.20334190e+11,
        6.16407568e+10,  2.69619707e+11,  3.16689576e+11])

In [94]:
regr_poly_L2_1.intercept_

array([1.89810066e+10])

In [96]:
preds_train_poly_L2_1 = regr_poly_L2_1.predict(X_train_poly_L2_1)
preds_test_poly_L2_1 = regr_poly_L2_1.predict(X_test_poly_L2_1)

In [97]:
np.sqrt(mean_squared_error(Y_train_poly_L2_1, preds_train_poly_L2_1)), np.sqrt(mean_squared_error(Y_test_poly_L2_1, preds_test_poly_L2_1))

(497094928399916.9, 529331148128982.75)

Alpha = 0.1

In [104]:
X_train_poly_L2_2, X_test_poly_L2_2, Y_train_poly_L2_2, Y_test_poly_L2_2 = train_test_split(X_poly_2, Y, test_size = 0.2, random_state = 1)

regr_poly_L2_2 = SGDRegressor(penalty='L2', alpha=0.1 / X_train_poly_L1_2.shape[0], tol=None, max_iter=100000)

regr_poly_L2_2.fit(X_train_poly_L2_2, Y_train_poly_L2_2)

SGDRegressor(alpha=9.345794392523364e-12, max_iter=100000, penalty='L2',
             tol=None)

In [105]:
regr_poly_L2_2.coef_

array([ 1.66145194e+11, -2.09433422e+10, -2.48046472e+10, -2.81599170e+11,
       -1.41035468e+10,  1.78985636e+11,  2.16930927e+10, -3.68988915e+10,
       -5.08824992e+10, -1.91794577e+10,  6.63347370e+10,  9.27934516e+10,
       -2.09433422e+10,  3.00534673e+10, -2.40585851e+10,  5.22680542e+10,
       -1.78712717e+11,  9.99598527e+10,  2.60570412e+11, -1.15243809e+11,
       -2.31810831e+11,  1.96297592e+10,  8.91013591e+10, -1.58968946e+11,
       -1.41035468e+10, -6.96754343e+10,  1.47861762e+11])

In [106]:
regr_poly_L2_2.intercept_

array([-2.89213702e+10])

In [107]:
preds_train_poly_L2_2 = regr_poly_L2_2.predict(X_train_poly_L2_2)
preds_test_poly_L2_2 = regr_poly_L2_2.predict(X_test_poly_L2_2)

In [108]:
np.sqrt(mean_squared_error(Y_train_poly_L1_2, preds_train_poly_L1_2)), np.sqrt(mean_squared_error(Y_test_poly_L1_2, preds_test_poly_L1_2))

(7.565594694815124e+43, 7.658280430856996e+43)

Alpha = 1

In [109]:
X_train_poly_L2_3, X_test_poly_L2_3, Y_train_poly_L2_3, Y_test_poly_L2_3 = train_test_split(X_poly_2, Y, test_size = 0.2, random_state = 1)

regr_poly_L2_3 = SGDRegressor(penalty='L2', alpha=1 / X_train_poly_L2_3.shape[0], tol=None, max_iter=1000)

regr_poly_L2_3.fit(X_train_poly_L2_3, Y_train_poly_L2_3)

SGDRegressor(alpha=0.0009345794392523365, penalty='L2', tol=None)

In [110]:
regr_poly_L2_3.coef_

array([-2.02205459e+11,  6.86871279e+10, -1.34764672e+11,  8.56726426e+10,
        6.64997789e+10, -1.22518766e+11, -1.99010216e+11, -2.28208477e+11,
       -8.18181006e+11, -1.47327029e+11,  5.85920157e+08, -8.30097537e+11,
        6.86871279e+10,  4.06216539e+11, -3.48420996e+10,  4.61637668e+10,
        5.72152289e+10,  6.55650478e+11, -2.21797478e+11,  6.52127104e+11,
       -9.93337008e+11,  8.41073278e+11,  1.89056254e+10, -5.00699113e+08,
        6.64997789e+10, -6.12158177e+10, -1.71913694e+11])

In [111]:
regr_poly_L2_3.intercept_

array([5.6491992e+09])

In [112]:
preds_train_poly_L2_3 = regr_poly_L2_3.predict(X_train_poly_L2_3)
preds_test_poly_L2_3 = regr_poly_L2_3.predict(X_test_poly_L2_3)

In [113]:
np.sqrt(mean_squared_error(Y_train_poly_L2_3, preds_train_poly_L2_3)), np.sqrt(mean_squared_error(Y_test_poly_L2_2, preds_test_poly_L2_3))

(1003815221419766.4, 1043436121294086.0)

Ya aqui se pudo observar que la regularización en general no esta ayudando a disminuir el error, sino todo lo contrario


**Respuestas**
1. Si fue posible disminuir el error de training utilizando una transformación a los features de grado 2 y 10, aunque en el de grado 10 disminuyo mas, fue a costo de que el error de testing aumentara un monton por lo cual se puede decir que estaba produciendoce overfitting, por lo tanto los hiperparametros que generaron el mejor modelo fue una transformación de grado 2 y sin usar regularización. A pesar de esto es un error muy alto y se deberia considerar cambiarse de un modelo lineal.
2. En el caso de este modelo pareciera que el factor más influyente es el de si la persona es fumadora, puesto a que es la que tiene el peso más alto de la regresión con 23000, seguido de lejos de numero de hijos con 400.